In [1]:
!pip install geopandas

     |████████████████████████████████| 972kB 5.1MB/s 
     |████████████████████████████████| 14.8MB 244kB/s 
     |████████████████████████████████| 6.5MB 39.5MB/s 


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
from urllib.request import urlopen
import json

In [3]:
df = pd.read_csv("/content/drive/MyDrive/SIPA CTO Fighting Digital Redlining Capstone Workshop/Research Notes & Docs/Data Analysis/Citywide_Public_Computer_Centers.csv")

In [4]:
df = df.rename(columns={"NTA Name": "neighborhood", "NTA Code": "ntacode"})

In [5]:
neighbor = gpd.read_file('/content/drive/MyDrive/SIPA CTO Fighting Digital Redlining Capstone Workshop/Research Notes & Docs/Data Analysis/neighborhood.json')

In [6]:
nta = gpd.read_file('/content/drive/MyDrive/SIPA CTO Fighting Digital Redlining Capstone Workshop/Research Notes & Docs/Data Analysis/nta.geojson')

In [7]:
nta

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005019.048,047,Borough Park,39247.2278309,Brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,QN51,52488277.5915,081,Murray Hill,33266.9048721,Queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,QN27,19726845.5925,081,East Elmhurst,19816.7121316,Queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,BK95,14522603.5915,047,Erasmus,18756.7018646,Brooklyn,3,"MULTIPOLYGON (((-73.94826 40.63861, -73.94923 ..."
4,QN07,22887772.8087,081,Hollis,20976.3355996,Queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
...,...,...,...,...,...,...,...,...
190,QN29,32675243.7117,081,Elmhurst,36711.3124169,Queens,4,"MULTIPOLYGON (((-73.86791 40.74493, -73.86788 ..."
191,QN50,21927768.8528,081,Elmhurst-Maspeth,32362.8010056,Queens,4,"MULTIPOLYGON (((-73.89175 40.74681, -73.89139 ..."
192,SI24,184878922.17,085,Todt Hill-Emerson Hill-Heartland Village-Light...,75943.5563395,Staten Island,5,"MULTIPOLYGON (((-74.09777 40.61062, -74.09730 ..."
193,SI36,67011968.2491,085,Old Town-Dongan Hills-South Beach,41973.1357654,Staten Island,5,"MULTIPOLYGON (((-74.06503 40.58899, -74.06503 ..."


In [8]:
point_gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))

In [9]:
geo_df = nta.merge(df, on = "ntacode")

In [10]:
#what percentage of locations have wifi?
df['Wi-Fi Available'].value_counts(normalize = True) * 100

True     87.795276
False    12.204724
Name: Wi-Fi Available, dtype: float64

In [11]:
#what are the access requirements for each location?
df['Access Requirements'].value_counts()

Must be 60+ years of age to access                                                                                                                                                                                                                                                                                   159
All ages; library card required for use of public computers                                                                                                                                                                                                                                                          151
All ages; library card required for use of public computers and WiFi                                                                                                                                                                                                                                                  74
Between 3pmand 6pm during the school year, and 8am and 6pm du

In [12]:
df['Oversight Agency'].value_counts()

Department for the Aging                         159
New York Public Library                           92
Queens Library                                    74
Department of Youth and Community Development     70
Brooklyn Public Library                           59
Department of Parks and Recreation                33
New York City Housing Authority                   21
Name: Oversight Agency, dtype: int64

In [13]:
df.shape

(508, 71)

In [14]:
#where are the locations by borough?
df['Borough'].value_counts()

Brooklyn         143
Queens           128
Manhattan        117
Bronx             95
Staten Island     25
Name: Borough, dtype: int64

In [15]:
#where are the locations by borough and neighborhood?
df.groupby('Borough')['neighborhood'].value_counts()

Borough        neighborhood                                   
Bronx          Mott Haven-Port Morris                             7
               Bronxdale                                          5
               Melrose South-Mott Haven North                     5
               Morrisania-Melrose                                 5
               Soundview-Castle Hill-Clason Point-Harding Park    5
                                                                 ..
Staten Island  New Brighton-Silver Lake                           1
               New Dorp-Midland Beach                             1
               Old Town-Dongan Hills-South Beach                  1
               Rossville-Woodrow                                  1
               Westerleigh                                        1
Name: neighborhood, Length: 175, dtype: int64

In [16]:
df[['ntacode','Workstation #']].groupby(['ntacode']).sum().agg(['min', 'max', 'mean'])

,Workstation #
min,5.00
max,500.00
mean,65.72


In [17]:
#which locations allow device loaning?
df['dummy'] = (df['Device Loaning'] > 0).astype(int)
df.groupby('Borough')['dummy'].value_counts(normalize = True) * 100

Borough        dummy
Bronx          0        100.00000
Brooklyn       0        100.00000
Manhattan      0        100.00000
Queens         0         50.78125
               1         49.21875
Staten Island  0        100.00000
Name: dummy, dtype: float64

In [18]:
df['dummy'].value_counts(normalize = True) * 100

0    87.598425
1    12.401575
Name: dummy, dtype: float64

In [19]:
px.set_mapbox_access_token('pk.eyJ1IjoiZWF0MjE1MyIsImEiOiJja2w0aXBtMWkwOXZuMm9wY3Z5Ymw4NnUzIn0.bUcUJMro4sQq85EOCUjn4A')

In [20]:
fig = px.scatter_mapbox(point_gdf,
                        lat = point_gdf.geometry.y,
                        lon = point_gdf.geometry.x,
                        hover_name="Location Name",
                        center = {"lat": 40.7, "lon": -73.7},
                        mapbox_style = "carto-positron",
                        zoom = 8.75,
                        title = "Public Computer Centers Locations")
fig.show()

In [ ]:
geo_df

In [21]:
geo

fig = px.choropleth_mapbox(geo_df,
                           geojson = nta,
                           locations = geo_df.index,
                           color = "Workstation #",
                           hover_name = "neighborhood",
                           color_continuous_scale = "Viridis",
                           center = {"lat": 40.7, "lon": -73.7},
                           mapbox_style = "carto-positron",
                           zoom = 8.75,
                           title = "# Workstations Available by Neighborhood")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [22]:
fig = px.scatter_mapbox(point_gdf,
                        lat = point_gdf.geometry.y,
                        lon = point_gdf.geometry.x,
                        color = point_gdf["Device Loaning"],
                        hover_name="Location Name",
                        center = {"lat": 40.7, "lon": -73.7},
                        mapbox_style = "carto-positron",
                        zoom = 8.75,
                        title = "Locations That Loan Devices")
fig.show()

In [23]:
fig = px.scatter_mapbox(point_gdf[point_gdf["Wi-Fi Available"] == True],
                        lat = point_gdf[point_gdf["Wi-Fi Available"] == True].geometry.y,
                        lon = point_gdf[point_gdf["Wi-Fi Available"] == True].geometry.x,
                        color = "Wi-Fi Available",
                        hover_name="Location Name",
                        center = {"lat": 40.7, "lon": -73.7},
                        mapbox_style = "carto-positron",
                        zoom = 8.75,
                        title = "Wifi Availability")
fig.show()